In [21]:
import os
import torch
from PIL import Image
import torchvision.transforms as transforms
import mlflow.pytorch

# --- Configuration ---
# Set the MLflow model URI.
# If you used the MLflow Model Registry, you can load the model by its registered name and stage.
# For example, loading the production model:
MODEL_URI = "models:/YOLOv8_TorchScript_Model/3"

# Alternatively, if you want to load from a specific run's artifact, uncomment the following line:
# MODEL_URI = "runs:/<run_id>/model_artifact"

# Define the expected input size for your model.
# YOLOv8 models are often trained on 640x640 images, but adjust if needed.
INPUT_SIZE = (640, 640)

# Define the device for inference.
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# --- Preprocessing ---
# Create a preprocessing pipeline. This converts the PIL image to a tensor,
# resizes it to the required dimensions, and scales pixel values to [0, 1].
preprocess = transforms.Compose([
    transforms.Resize(INPUT_SIZE),    # Resize image to match model input size
    transforms.ToTensor(),            # Convert image to tensor and scale pixel values
    # If your model requires normalization, add:
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


def load_model(model_uri: str) -> torch.nn.Module:
    """
    Load a TorchScript model logged in MLflow.

    Args:
        model_uri (str): MLflow model URI.

    Returns:
        torch.nn.Module: Loaded TorchScript model ready for inference.
    """
    # Load the model using MLflow's PyTorch API
    model = mlflow.pytorch.load_model(model_uri)
      # Move the model to the appropriate device
    model.eval()      # Set the model to evaluation mode
    return model


def predict_image(model: torch.nn.Module, image_path: str):
    """
    Perform inference on an image using the loaded model.

    Args:
        model (torch.nn.Module): The loaded TorchScript YOLOv8 model.
        image_path (str): Path to the input image.

    Returns:
        The raw output from the model. Postprocessing might be required depending on your export.
    """
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image file not found at: {image_path}")

    # Load and preprocess the image
    image = Image.open(image_path).convert("RGB")
    input_tensor = preprocess(image)
    input_tensor = input_tensor.unsqueeze(0)  # Add batch dimension

    # Move input tensor to the correct device

    # Perform inference without tracking gradients
    with torch.no_grad():
        outputs = model(input_tensor)

    # Note: The output structure will depend on how the TorchScript export was performed.
    # YOLOv8 exports might include integrated postprocessing (NMS, etc.) or might return raw predictions.
    return outputs


# --- Main Execution ---
if __name__ == "__main__":
    # Specify the path to the input image
    image_path = "../test_image/000000000328.jpg"

    # Load the model
    model = load_model(MODEL_URI)

    # Run prediction on the provided image
    predictions = predict_image(model, image_path)

    # Print or further process the predictions
    print("Raw predictions:", predictions)
    print(predictions.shape)
    # Optionally: Add postprocessing here if your export does not include NMS or threshold filtering.


c:\Users\Admin\anaconda3\Lib\site-packages\torch\serialization.py:1328: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn(


Raw predictions: tensor([[[2.9184e+00, 2.4811e+01, 2.7877e+01,  ..., 4.9787e+02,
          5.7356e+02, 5.7950e+02],
         [3.3195e+00, 2.6033e+00, 2.4984e+00,  ..., 5.6190e+02,
          5.6086e+02, 5.6282e+02],
         [5.6160e+00, 4.5851e+01, 5.1095e+01,  ..., 2.8147e+02,
          1.2626e+02, 1.2822e+02],
         ...,
         [1.6302e-07, 2.9225e-07, 4.4020e-07,  ..., 1.0410e-06,
          1.2776e-06, 2.2958e-06],
         [1.1281e-07, 1.7772e-07, 1.9181e-07,  ..., 1.2685e-06,
          1.3668e-06, 1.5561e-06],
         [4.1009e-07, 4.6186e-07, 4.9135e-07,  ..., 1.4077e-06,
          1.3306e-06, 1.3260e-06]]])
torch.Size([1, 84, 8400])


In [11]:
output.shape

torch.Size([1, 84, 8400])

In [11]:
import mlflow
import pandas as pd

# ---------------------------
# Configuration
# ---------------------------
mlflow.set_tracking_uri("http://localhost:8080")
target_run_name = "YOLOv8_v1"
artifact_path = "model_weights/yolov8n.pt"  # The path to your artifact within the run

# ---------------------------
# Step 1: Search for the Run by Name
# ---------------------------
# Run names are stored as a tag "mlflow.runName". Filter runs based on that.
run_id = mlflow.search_runs(search_all_experiments=True)['run_id'][0]
print(run_id)
# If multiple runs are found, select the most recent run using the 'start_time' column.

downloaded_path = mlflow.artifacts.download_artifacts(
    run_id=run_id,
    artifact_path=artifact_path
)

print(f"Artifact downloaded to: {downloaded_path}")


842e4e9c47e84479841abaae65f74b0e


Artifact downloaded to: C:\Users\Admin\AppData\Local\Temp\tmpg2opy04u\model_weights/yolov8n.pt


In [12]:
from ultralytics import YOLO
model = YOLO(downloaded_path)

c:\Users\Admin\anaconda3\Lib\site-packages\ultralytics\nn\tasks.py:781: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")


In [13]:
import cv2

result = model('../test_image/000000000328.jpg')


image 1/1 c:\Users\Admin\Desktop\try\mlflow\..\test_image\000000000328.jpg: 512x640 3 persons, 1 bench, 1 backpack, 2 ties, 1 book, 37.6ms
Speed: 7.1ms preprocess, 37.6ms inference, 108.5ms postprocess per image at shape (1, 3, 512, 640)


In [14]:
result[0].boxes

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([ 0.,  0.,  0., 73., 27., 27., 24., 13.], device='cuda:0')
conf: tensor([0.8925, 0.8672, 0.8569, 0.6029, 0.5812, 0.3420, 0.2933, 0.2565], device='cuda:0')
data: tensor([[2.0172e+02, 9.0509e+01, 4.0114e+02, 4.7412e+02, 8.9248e-01, 0.0000e+00],
        [4.5097e+01, 6.6411e+01, 2.6366e+02, 4.0951e+02, 8.6716e-01, 0.0000e+00],
        [3.3829e+02, 6.8681e+01, 5.8670e+02, 4.8304e+02, 8.5687e-01, 0.0000e+00],
        [2.4880e+02, 2.2124e+02, 3.3245e+02, 2.6056e+02, 6.0289e-01, 7.3000e+01],
        [3.2016e+02, 1.7012e+02, 3.3753e+02, 2.3333e+02, 5.8116e-01, 2.7000e+01],
        [4.6084e+02, 1.7738e+02, 4.7902e+02, 2.2015e+02, 3.4198e-01, 2.7000e+01],
        [1.6073e+01, 1.5715e+02, 1.3660e+02, 2.4961e+02, 2.9330e-01, 2.4000e+01],
        [2.0226e+02, 1.5518e+02, 5.8856e+02, 4.7754e+02, 2.5653e-01, 1.3000e+01]], device='cuda:0')
id: None
is_track: False
orig_shape: (491, 640)
shape: torch.Size([8, 6])
xywh: tensor([[301.42